In [1]:
import os 
import nltk 
import wave 
from glob import glob 
import pandas as pd

In [2]:
def remove_punct(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+") 
    new_words = tokenizer.tokenize(text)
    t = ''
    for i in new_words: 
        t+= i+' '
    return t 

In [3]:
sessions = glob("/home/csikasote/Documents/Unza-NLP-Repos/zambezi-voice/nyanja/wav/*/session*")
linkers = glob("/home/csikasote/Documents/Unza-NLP-Repos/zambezi-voice/nyanja/wav/*/*linker.txt")

In [4]:
print(len(linkers))
print(len(sessions))

15
15


In [5]:
all_session_text = open('all_sessions.txt', 'w') 

In [6]:
corrupted_list = []
for linker, sess in zip(linkers, sessions):
    sess_read = open(sess, 'r')
    link_read = open(linker, 'r')
    for link, sentence in zip(link_read.readlines(), sess_read.readlines()):
        wav_name = os.path.basename(link).replace('\n','')
        wav_path = glob("/home/csikasote/Documents/Unza-NLP-Repos/zambezi-voice/nyanja/wav/*/{0}".format(wav_name))
        if len(wav_path)==0:
            continue
        wav_path = wav_path[0]
        try: 
            w = wave.open(wav_path, 'r') 
            d = w.readframes(w.getnframes()) 
        except: 
            #print('corrupted audio: {0} '.format(wav_name))
            corrupted_list.append(wav_name)
            # uncomment if you want to delete the corrupted file 
            #os.remove(wav_path)
            continue 
        indices = '' 
        sentence = sentence.replace('##', '') 
        sentence = remove_punct(sentence).lower()        
        all_session_text.writelines(wav_name[:-4] + '\t' + sentence + '\n')
        
corrupt_audio_list_df = pd.DataFrame(corrupted_list, columns =['audio_id'])

nyanja_data_df = pd.read_csv('./all_sessions.txt', delimiter='\t', header=None, names=['audio_id', 'sentence'])
nyanja_data_df.to_csv('./data.csv', sep='\t',index=False)

In [7]:
corrupt_audio_list_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   audio_id  0 non-null      object
dtypes: object(1)
memory usage: 0.0+ bytes


In [8]:
nyanja_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   audio_id  713 non-null    object
 1   sentence  712 non-null    object
dtypes: object(2)
memory usage: 11.3+ KB


In [9]:
nyanja_data_df.audio_id

0       220919-073442_nya_ddf_elicit_0
1       220919-073442_nya_ddf_elicit_1
2       220919-073442_nya_ddf_elicit_2
3       220919-073442_nya_ddf_elicit_3
4       220919-073442_nya_ddf_elicit_4
                    ...               
708    220917-063641_nya_ddf_elicit_23
709    220917-063641_nya_ddf_elicit_24
710    220917-063641_nya_ddf_elicit_25
711    220917-063641_nya_ddf_elicit_26
712          220917-063641_nya_ddf_eli
Name: audio_id, Length: 713, dtype: object

In [10]:
corrupt_audio_list_df.audio_id

Series([], Name: audio_id, dtype: object)